# Demo SingleRun
This notebook shows a single run consisting of the following four phases:
* sail empty
* loading
* sail full
* unloading

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

## Definition of Site

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5d",  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 6,
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5c",  # For logging purposes
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 6,  # The capacity of the site
    "level": 0,
}  # The actual volume of the site (empty of course)

# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

init
level: 6
completed init
init
level: 0
completed init


## Definition of Vessels

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
    ),
    {},
)

# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 4,  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}

hopper = TransportProcessingResource(**data_hopper)

init
level: 0
completed init


## Definition of process

In [4]:
single_run = []

move_activity_to_harbor_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing empty",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5d",  # For logging purposes
    "registry": registry,
    "mover": hopper,
    "destination": from_site,
    "postpone_start": True,
}
single_run.append(model.MoveActivity(**move_activity_to_harbor_data))

shift_amount_activity_loading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer MP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 4,
    "duration": 10,
    "postpone_start": True,
}
single_run.append(model.ShiftAmountActivity(**shift_amount_activity_loading_data))

move_activity_to_site_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing filler",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "mover": hopper,
    "destination": to_site,
    "postpone_start": True,
}
single_run.append(model.MoveActivity(**move_activity_to_site_data))

shift_amount_activity_unloading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer TP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff54",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": hopper,
    "destination": to_site,
    "amount": 4,
    "duration": 10,
    "postpone_start": True,
}
single_run.append(model.ShiftAmountActivity(**shift_amount_activity_unloading_data))

basic_activity_data = {
    "env": my_env,
    "name": "Basic activity",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5h",  # For logging purposes
    "registry": registry,
    "duration": 0,
     "additional_logs": [hopper],
    "postpone_start": True,
}
single_run.append(model.BasicActivity(**basic_activity_data))


sequential_activity_data = {
    "env": my_env,
    "name": "Single run process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
}
activity = model.SequentialActivity(**sequential_activity_data)

expr = [{"type":"container", "concept": to_site, "state":"full"}]
while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "while",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5g",  # For logging purposes
    "registry": registry,
    "sub_process": activity,
    # "condition_event": [from_site.container.get_empty_event, to_site.container.get_full_event],
    #"condition_event": to_site.container.get_full_event,
    "condition_event": expr,
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


while Activity keep_resources []
get_full_event : default
start get_available
start event instance None


In [5]:
my_env.run()


conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{'default': {6: <Event() object at 0x251c81b1a08>}}
amount :6
put_callback - id_ default
{}
Mover_move after mover resource request
Mover_move after move
Activity end(sailing empty)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.6dbbbdf4-4589-11e9-a501-b469212bff5d': 6, 'destination.6dbbbdf6-4589-11e9-95a2-b469212bff5b': 4}
destination request : {<simpy.resources.resource.Resource object at 0x00000251C81A9488>: <Request() object at 0x251c799b948>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x00000251C81A9488>: <Request() object at 0x251c799b948>}
site request : {<simpy.resources.resource.Resource object at 0x00000251C81A9488>: <Request() object at 0x251c799b948>, <simpy.r

In [6]:
log_df = pd.DataFrame(hopper.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data = data.drop_duplicates()
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,sailing empty,START,1970-01-01 01:00:00.000000,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5d
1,sailing empty,STOP,1970-01-01 01:00:00.000000,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5d
2,transfer default to Hopper 01,START,1970-01-01 01:00:00.000000,4,6dbbbdf7-4589-11e9-bf3b-b469212bff52
4,transfer default to Hopper 01,STOP,1970-01-01 01:00:10.000000,4,6dbbbdf7-4589-11e9-bf3b-b469212bff52
6,sailing filled,START,1970-01-01 01:00:10.000000,4,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
7,sailing filled,STOP,1970-01-01 01:15:52.824591,4,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
8,transfer default to Dumplocatie,START,1970-01-01 01:15:52.824591,4,6dbbbdf7-4589-11e9-bf3b-b469212bff54
10,transfer default to Dumplocatie,STOP,1970-01-01 01:16:02.824591,4,6dbbbdf7-4589-11e9-bf3b-b469212bff54
12,Basic activity,START,1970-01-01 01:16:02.824591,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5h
13,Basic activity,STOP,1970-01-01 01:16:02.824591,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5h
